In [1]:
import tensorflow as tf
import numpy as np
import glob
import os
import sys
import pandas as pd

In [2]:
imagePath = '/home/adam/Desktop/Fish/test_stg1/img_07678.jpg'
modelFullPath = '/home/adam/Fish/models/inc_retrain_full/output_graph.pb'
labelsFullPath = '/home/adam/Fish/models/inc_retrain_full/output_labels.txt'

In [6]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')


In [7]:
def run_inference_on_image():
    answer = None

    if not tf.gfile.Exists(imagePath):
        tf.logging.fatal('File does not exist %s', imagePath)
        return answer

    image_data = tf.gfile.FastGFile(imagePath, 'rb').read()

    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:

        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        predictions = sess.run(softmax_tensor,
                               {'DecodeJpeg/contents:0': image_data})
        predictions = np.squeeze(predictions)

        top_k = predictions.argsort()[-5:][::-1]  # Getting top 5 predictions
        f = open(labelsFullPath, 'rb')
        lines = f.readlines()
        labels = [str(w).replace("\n", "") for w in lines]
        for node_id in top_k:
            human_string = labels[node_id]
            score = predictions[node_id]
            print('%s (score = %.5f)' % (human_string, score))

        answer = labels[top_k[5]]
        return answer

In [8]:
if __name__ == '__main__':
    run_inference_on_image()

InvalidArgumentError: NodeDef mentions attr 'dct_method' not in Op<name=DecodeJpeg; signature=contents:string -> image:uint8; attr=channels:int,default=0; attr=ratio:int,default=1; attr=fancy_upscaling:bool,default=true; attr=try_recover_truncated:bool,default=false; attr=acceptable_fraction:float,default=1>; NodeDef: DecodeJpeg = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_DecodeJpeg/contents_0)
	 [[Node: DecodeJpeg = DecodeJpeg[acceptable_fraction=1, channels=3, dct_method="", fancy_upscaling=true, ratio=1, try_recover_truncated=false, _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_DecodeJpeg/contents_0)]]

Caused by op 'DecodeJpeg', defined at:
  File "/home/adam/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/adam/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 